In [1]:
!pip install open3d
import pydicom
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import numpy as np
import cv2  
import operator 
from collections import Counter
import open3d as o3d
import pyvista as pv
from pyvista import examples
import pandas as pd

ERROR: Could not find a version that satisfies the requirement open3d (from versions: none)
ERROR: No matching distribution found for open3d


ModuleNotFoundError: No module named 'open3d'

In [ ]:
class imagen:
    def __init__ (self, slice, time, img, info, dir, imgposition, pxspacingx, pxspacingy):
        self.slice = slice
        self.time = time
        self.img = img
        self.info = info
        self.dir = dir 
        self.imgposition = imgposition
        self.pxspacingx = pxspacingx
        self.pxspacingy = pxspacingy


In [ ]:
def position_ROI(Dcm_im_ROI):
    '''
    Give the general position of the ROI used to be copy on every image
    '''
    Dcm_im8 = (Dcm_im_ROI).astype(np.uint8)
    Dcm_im8 = cv2.applyColorMap(Dcm_im8,cv2.COLORMAP_BONE)
    showCrosshair = False
    r = cv2.selectROI("Seleccione el ROI a analizar", Dcm_im8,showCrosshair)
    cv2.destroyAllWindows()
    # r = top x, top y, bottom x, bottom y
    return r

In [ ]:
def demo_crop_geometry(pcd):
    print("Demo for manual geometry cropping")
    print(
        "1) Press 'Y' twice to align geometry with negative direction of y-axis"
    )
    print("2) Press 'K' to lock screen and to switch to selection mode")
    print("3) Drag for rectangle selection,")
    print("   or use ctrl + left click for polygon selection")
    print("4) Press 'C' to get a selected geometry and to save it")
    print("5) Press 'F' to switch to freeview mode")
    pcd = pcd
    o3d.visualization.draw_geometries_with_editing([pcd])

In [ ]:
image_list, image_slices, pmesh, img_qtt, countmesh, result, dcm_im_all = [], [], [], 0, 0, 0, []
# Carga las imágenes, genera lista de slice y genera objetos
path = tk.Tk()
path.withdraw()
file_name = filedialog.askopenfilenames()

In [ ]:
for i in file_name:
    Dcm = pydicom.dcmread(i)
    # Filtrado de imágenes de magnitud
    if str(Dcm[0x2005, 0x116e].value) == 'PCA': 
        image_list.append('imagen' + str(img_qtt)) # Lista de imágenes
        image_slices.append(Dcm[0x2001,0x100a].value)
        dir = 0
        # Filtrado según componentes
        if Dcm[0x2001, 0x101a].value[0]>50 and Dcm[0x2001, 0x101a].value[1]>50 and Dcm[0x2001, 0x101a].value[2]>50: # Imagen neta
            dir = 1
        elif Dcm[0x2001, 0x101a].value[0]>50 and Dcm[0x2001, 0x101a].value[1]==0 and Dcm[0x2001, 0x101a].value[2]==0: # RL # X
            dir = 2
        elif Dcm[0x2001, 0x101a].value[0]==0 and Dcm[0x2001, 0x101a].value[1]>50 and Dcm[0x2001, 0x101a].value[2]==0: # AP # Y
            dir = 3
        elif Dcm[0x2001, 0x101a].value[0]==0 and Dcm[0x2001, 0x101a].value[1]==0 and Dcm[0x2001, 0x101a].value[2]>50: # FH # Z
            dir = 4
        image_list[img_qtt] = imagen (Dcm[0x2001,0x100a].value, Dcm[0x0018,0x1060].value, np.copy(Dcm.pixel_array), Dcm, dir, Dcm[0x0020,0x0032].value, Dcm[0x0028,0x0030].value[0], Dcm[0x0028,0x0030].value[1]) # Lista de objetos tipo image
        dcm_im_all.append(Dcm.pixel_array)
        img_qtt = img_qtt + 1

In [ ]:
# Ordena los objetos y calcula variables
sorted_image_list  = sorted(image_list, key = operator.attrgetter('dir', 'slice', 'time')) 
imgslctr = Counter(image_slices) # Número de slice : Cantidad de imágenes
slice_qtt = max(imgslctr.keys()) # Cantidad de slices
sl_n_imgs = int(max(imgslctr.values())/4) # Cantidad de imágenes por Slice
img_by_type = int(img_qtt/4)
height, width = (image_list[0].img).shape # Tamaño de las imágenes

In [ ]:
# Matrices a trabajar 
maskmatrix = np.zeros((slice_qtt,height,width), np.uint16)    # Máscara de todos los slice  
maskimage = np.zeros((height,width), np.uint16)               # Máscara unitaria
maskimageROI = np.zeros((height,width), np.uint64)            # Máscara unitaria para ROI
maskimgmatrix = np.zeros((slice_qtt,height,width), np.uint16) # Máscara pos-ROI de todos los slice
reconstucted_matrix = np.zeros((height,width), np.uint16)

In [ ]:
# Genera el Threshold y plasma en un arreglo toda la info (en ese arreglo se va a realizar el ROI)
# Verifica que exista info en el slice anterior para eliminar vasos o info no deseada
for q in range(slice_qtt): 
    img_org = sorted_image_list[q * sl_n_imgs].img 
    prev_img_org = sorted_image_list[(q-1) * sl_n_imgs].img
    for j in range(height): 
        for k in range(width):
            if img_org[j,k] <= 80: # 500 para FFE
                maskimage[j,k] = 0
                maskimageROI[j,k] = maskimageROI[j,k] + 0
            else:
                if q == 0:
                    maskimage[j,k]= 255
                    maskimageROI[j,k] = maskimageROI[j,k] + 128
                else:
                    # Comparacion con el slice anterior (para todos menos para q=0)
                    for o in range(3):
                        for p in range(3):
                            if prev_img_org[j+o-1, k+p-1] == 255:
                                result = result + 1
                            else:
                                result = result
                    if result>0:
                        maskimage[j+o-1, k+p-1] = 255
                        maskimageROI[j+o-1, k+p-1] = maskimageROI[j+o-1, k+p-1] + 128
                    else:
                        maskimage[j+o-1, k+p-1] = 0
                        maskimageROI[j+o-1, k+p-1] = maskimageROI[j+o-1, k+p-1]
    
# Se realiza el ROI y se elimina la info fuera de él esto es para el slice maximo (me quedaba fuera de rango)
    if q == (slice_qtt-1):
        utROI = position_ROI(maskimageROI)
        for l in range(width):
            for m in range(height):
                if m>float(utROI[0]) and m<float(utROI[0]+utROI[2]) and l>float(utROI[1]) and l<float(utROI[1]+utROI[3]):
                    maskimage[l,m]=maskimage[l,m]
                else:
                    maskimage[l,m] = 0 
    maskimgmatrix[q] = maskimage

In [ ]:
plt.imshow(maskimgmatrix[slice_qtt-1])

In [ ]:
# hago lo mismo que hacia en las lineas anteriores pero para los demas slices

for q in range(slice_qtt):
    for l in range(width):
            for m in range(height):
                if m>float(utROI[0]) and m<float(utROI[0]+utROI[2]) and l>float(utROI[1]) and l<float(utROI[1]+utROI[3]):
                    maskimgmatrix[q,l,m]=maskimgmatrix[q,l,m]
                else:
                    maskimgmatrix[q,l,m] = 0

In [ ]:
# Se realiza la deteccion de bordes con sobel
for q in range(slice_qtt):
    maskimageinv = cv2.flip(maskimgmatrix[q],-1)
    sobelx = cv2.Sobel(maskimgmatrix[q],-1,1,0,ksize=1) # Image , DDepth = -1 (uint8) would be the result
    sobely = cv2.Sobel(maskimgmatrix[q],-1,0,1,ksize=1) # 1 , 0  for dx   and  0 , 1 for dy
    sobeldir = sobelx + sobely
    sobelxinv = cv2.Sobel(maskimageinv,-1,1,0,ksize=1) 
    sobelyinv = cv2.Sobel(maskimageinv,-1,0,1,ksize=1)
    sobelinv = sobelxinv + sobelyinv 
    reconstucted_matrix = cv2.flip(sobelinv,-1)
    sobel = sobeldir +  reconstucted_matrix
    maskmatrix[q] = sobel

    # Seleccion de los valores internos a los bordes 
    for i in range(height):
        for j in range (width):
            if (j > 1) and (i > 1): # FIJARME DE OPTIMIZARLO
                if (maskmatrix[q][i][j] == 0) and (maskmatrix[q][i-1][j] != 0) and (maskmatrix[q][i][j-1] != 0):
                    maskmatrix[q][i][j] = 200 # 200 por poner un valor que sea exclusivo de los pixeles internos
    
print(maskmatrix)

In [ ]:
# plt.imshow(maskmatrix[0])
# plt.imshow(maskmatrix[3])
plt.imshow(maskmatrix[10])

In [ ]:
for i in range(slice_qtt):
    for j in range(height):
        for k in range(width):
            if maskmatrix[i][j][k] > 0:
                print('hola')

In [ ]:
plt.imshow(maskmatrix[1])

In [ ]:
sorted_image_list[1 * sl_n_imgs].imgposition[2]

In [ ]:
# COPIA COORDENADAS MM Se obtienen las coordenadas limpias de cada uno de los puntos de valor alto post filtro sobel
slice_points, slice_cells, coordy, coordx = [], [], 0, 0

for q in range (slice_qtt):
    # ESTA LINEA ES LO MAS NEFASTO DEL UNIVERSO PERO NECESITO SABER
    if sorted_image_list[q * sl_n_imgs].imgposition[2] >= -44.19343 and sorted_image_list[q * sl_n_imgs].imgposition[2] <= -31.0219: #VALORES SACADOS DE PARAVIEW
        coordx = sorted_image_list[q * sl_n_imgs].imgposition[0]
        coordy = sorted_image_list[q * sl_n_imgs].imgposition[1]
        for j in range(height):
            for k in range(width):
                # print(utROI[3])
                if k > int(utROI[1]) and k < int(utROI[1] + utROI[3]) and j > int(utROI[0]) and j < int(utROI[0] + utROI[2]): # Ajustar al Roi
                    if maskmatrix[q][k][j] > 0:
                        countmesh += 1 
                        # TENER CUIDADO QUE ACA ESTOY EN UN SOLO TIEMPO (OBVIO EN EL ULTIMO)
                        pclean = [coordx, coordy, sorted_image_list[q * sl_n_imgs].imgposition[2]]
                        pmesh.append(pclean) # Podemos forzar etiquetas o podemos obtener etiquetas por defecto
                coordx = coordx + sorted_image_list[q * sl_n_imgs].pxspacingx
            coordy = coordy + sorted_image_list[q * sl_n_imgs].pxspacingy
                    # print(pclean)
    # slice_points.append(countmesh) 
    # slice_cells.append(countmesh//4)

print(pmesh)

In [ ]:
# Se obtienen las coordenadas limpias de cada uno de los puntos de valor alto post filtro sobel
slice_points, slice_cells = [], []
for q in range (slice_qtt):
    # ESTA LINEA ES LO MAS NEFASTO DEL UNIVERSO PERO NECESITO SABER
    if sorted_image_list[q * sl_n_imgs].imgposition[2] >= -44.19343 and sorted_image_list[q * sl_n_imgs].imgposition[2] <= -31.0219: #VALORES SACADOS DE PARAVIEW
        for j in range(height):
            for k in range(width):
                # print(utROI[3])
                if k > int(utROI[1]) and k < int(utROI[1] + utROI[3]) and j > int(utROI[0]) and j < int(utROI[0] + utROI[2]): # Ajustar al Roi
                    if maskmatrix[q][k][j] > 0:
                        countmesh += 1
                        pclean = [j, k, q]
                        pmesh.append(pclean) # Podemos forzar etiquetas o podemos obtener etiquetas por defecto
                        # print(pclean)


In [ ]:
print(pclean)

In [ ]:
# Generación de arreglo vectorial para cada punto y csv correspondiente
vector_array = np.ndarray((countmesh,3)) # Definir point_qtt
counter = 0 
for q in range(slice_qtt):
    # ESTA LINEA ES LO MAS NEFASTO DEL UNIVERSO PERO NECESITO SABER
    if sorted_image_list[q * sl_n_imgs].imgposition[2] >= -44.19343 and sorted_image_list[q * sl_n_imgs].imgposition[2] <= -31.0219: #VALORES SACADOS DE PARAVIEW
        for j in range(height): 
                for k in range(width): 
                    if k > float(utROI[1]) and k < float(utROI[1] + utROI[3]) and j > float(utROI[0]) and j < float(utROI[0] + utROI[2]): # Ajustar al Roi
                            if maskmatrix[q][k][j] > 0: 
                                vector_x = sorted_image_list[((q*sl_n_imgs)+img_by_type*1)].img[k,j]
                                vector_y = sorted_image_list[((q*sl_n_imgs)+img_by_type*2)].img[k,j]
                                vector_z = sorted_image_list[((q*sl_n_imgs)+img_by_type*3)].img[k,j]
                                vector_array[counter] = [vector_x, vector_y , vector_z]
                                counter += 1



In [ ]:
min = 1000000
for q in range(slice_qtt):
    # ESTA LINEA ES LO MAS NEFASTO DEL UNIVERSO PERO NECESITO SABER
    if sorted_image_list[q * sl_n_imgs].imgposition[2] < min:
        min = sorted_image_list[q * sl_n_imgs].imgposition[2]

print(min)


In [ ]:
print(sl_n_imgs)

In [ ]:
# Generación de arreglo vectorial para cada punto y csv correspondiente
countercsv = 0
for q in range(slice_qtt):
    # ESTA LINEA ES LO MAS NEFASTO DEL UNIVERSO PERO NECESITO SABER
    if sorted_image_list[q * sl_n_imgs].imgposition[2] == min: #VALORES SACADOS DE PARAVIEW
        for ph in range(sl_n_imgs):
            counter = 0
            # print(ph)
            vector_array = np.ndarray((countmesh,3)) # Definir point_qtt
            for j in range(height):
                    for k in range(width):
                        if k > float(utROI[1]) and k < float(utROI[1] + utROI[3]) and j > float(utROI[0]) and j < float(utROI[0] + utROI[2]): # Ajustar al Roi
                                if maskmatrix[q][k][j] > 0: 
                                    vector_x = sorted_image_list[((q*(ph+1))+img_by_type*1)].img[k,j]
                                    vector_y = sorted_image_list[((q*(ph+1))+img_by_type*2)].img[k,j]
                                    vector_z = sorted_image_list[((q*(ph+1))+img_by_type*3)].img[k,j]
                                    vector_array[counter] = [vector_x, vector_y , vector_z]
                                    counter += 1
            csvname =  "C:\\Users\\mpaul\\Documents\\FUESMEN\\phasesOpenFoam\\" + str(countercsv) + ".csv" 
            hola = pd.DataFrame(vector_array)
            hola.to_csv(csvname)
            # print(csvname)
            countercsv += 1


In [ ]:
csvnamepmax =  "C:\\Users\\mpaul\\Documents\\FUESMEN\\phasesOpenFoam\\" + str(20-1) + ".csv" #(sl_n_imgs)
pMax = pd.read_csv(csvnamepmax, header=None, usecols=[1,2,3])
pMaxnp = pMax.to_numpy()
counterp = 0

for i in reversed(range(20)):
    csvnamep =  "C:\\Users\\mpaul\\Documents\\FUESMEN\\phasesOpenFoam\\" + str(i) + ".csv" 
    if i != 0:
        csvnamepprev =  "C:\\Users\\mpaul\\Documents\\FUESMEN\\phasesOpenFoam\\" + str(i-1) + ".csv" 
    pAct = pd.read_csv(csvnamep, header=None, usecols=[1,2,3])
    pActnp = pAct.to_numpy()
    pPrev = pd.read_csv(csvnamepprev, header=None, usecols=[1,2,3])
    # print(pActnp)
    pPrevnp = pPrev.to_numpy()
    x = int(np.shape(pActnp)[0])
    y = int(np.shape(pActnp)[1])
    # print("x:", x, "y:", y)
    parray = np.ndarray((x,y))
    for k in range(y):
        for l in range(x):
            if i == 0:
                parray[l,k] = pActnp[l,k] - pMaxnp[l,k]
            else:
                parray[l,k] = pActnp[l,k] - pPrevnp[l,k]
            print(parray)
    csvnamep =  "C:\\Users\\mpaul\\Documents\\FUESMEN\\pressuresOpenFoam\\" + str(counterp) + ".csv" 
    holap = pd.DataFrame(parray)
    holap.to_csv(csvnamep, header=None, index=False)
    counterp += 1

            # print(csvname)
            


In [ ]:
numero = 1
string = "C:\\Users\\mpaul\\Documents\\FUESMEN" + str(numero)
print(string)

In [ ]:
# COPIA CON LA UBICACION CON RESPECTO A ISOCENTRO Generación de arreglo vectorial para cada punto 
vector_array = np.ndarray((countmesh,3)) # Definir point_qtt
counter = 0 
for q in range(slice_qtt):
    for j in range(height): 
            for k in range(width): 
                if k > float(utROI[1]) and k < float(utROI[1] + utROI[3]) and j > float(utROI[0]) and j < float(utROI[0] + utROI[2]): # Ajustar al Roi
                        if maskmatrix[q][k][j] > 0: 
                            vector_x = sorted_image_list[((q*sl_n_imgs)+img_by_type*1)].img[k,j]
                            vector_y = sorted_image_list[((q*sl_n_imgs)+img_by_type*2)].img[k,j]
                            vector_z = sorted_image_list[((q*sl_n_imgs)+img_by_type*3)].img[k,j]
                            vector_array[counter] = [vector_x, vector_y , vector_z]
                            counter += 1

In [ ]:
# ARMO LOS VECTORES EN PYVISTA
scalar_array = [10]*countmesh
points = np.asarray(pmesh)
point_cloud = pv.PolyData(points)

'''raul = point_cloud.reconstruct_surface(nbr_sz = 10)

pl = pv.Plotter(shape=(1,2))
pl.add_mesh(point_cloud)
pl.add_title('Point Cloud of 3D Surface')
pl.subplot(0,1)
pl.add_mesh(raul, color=True, show_edges=True)
pl.add_title('Reconstructed Surface')
pl.show()'''

# point_cloud['vectors'] = vector_array

# MESH
mesh = point_cloud.delaunay_3d(alpha=1.4) 
plotter = pv.Plotter()
actor = plotter.add_mesh(mesh, color='red')
plotter.show()

# glyphs = mesh.glyph(orient="vectors", scale=False, factor=0.9) #A LO MEJOR NOS TOQUE CAMBIAR EL FACTOR
# grid = glyphs.cast_to_unstructured_grid()

In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh, color="grey", ambient=0.6, opacity=0.1, show_edges=False)
pl.add_mesh(grid,  lighting=False, scalar_bar_args={'title': "Vector Magnitude"})
pl.show()

In [ ]:
# https://gitlab.onelab.info/gmsh/gmsh/-/issues/1000
"""ESTA LINEA ES LA QUE VA 14/4/22"""
import gmsh

gmsh.initialize()

i=0
for [x, y, z] in pmesh:
    gmsh.model.occ.addPoint(x, y, z, tag=i)
    # las lineas de addline funcionan, pero deberia hacer la limpieza antes
    # if i > 0:
    #     gmsh.model.occ.addLine(startTag= i-1, endTag = i)
    i = i+1

# only synchronize after adding all the points to the model
gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(1)
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)
gmsh.write("model" + ".msh")

gmsh.fltk.run()

gmsh.finalize()